<a href="https://colab.research.google.com/github/ArghyaPal/Zero-shot-task-transfer/blob/master/MNIST_classifier_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
jj=0
for jj in range(20):  # loop over the dataset multiple times

    import os
    import torch
    import torchvision
    import torch.nn as nn
    from torchvision import transforms
    from torchvision.utils import save_image
    from torch.autograd import Variable

    from torch.utils.data import DataLoader
    from torch.utils.data import Dataset

    import matplotlib.pyplot as plt
    import numpy as np
    seed = jj
    print("Seed Number Starting:")
    print(seed)
    print('\n')
    torch.manual_seed(seed)

    # Decide which device we want to run on
    ngpu = 4
    device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

    ## Hyperparameteres for this work
    # The noise dimension for the Generator, i.e. the z dimension, sampled from a Gaussian Distribution
    latent_size = 64
    # The network layer dimension of Generator and Discriminator
    hidden_size = 256
    # The generated image size/image size. MNIST size 28 X 28 = 784
    image_size = 784
    # The number of channels
    nchannel = 1
    # Number of epochs
    num_epochs = 3
    # A mini-batch size
    batch_size = 20
    # Adam learning rate
    lr = 0.0001

    # Number of classes
    nclass = 10          # For MNIST dataset, please change it for other dataset

    # Image processing
    transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.5),   # 3 for RGB channels
                                        std=(0.5))])


    # MNIST dataset
    mnist = torchvision.datasets.MNIST(root='./data/',
                                      train=True,
                                      transform=transform,
                                      download=True)

    # Class with label 0
    idx_other = mnist.train_labels == 8  # <---- Class 
    mnist.train_labels[idx_other] = 1

    # All other classses
    idx_other = mnist.train_labels != 8   # <---- Class
    mnist.train_data[idx_other]   = 0
    mnist.train_labels[idx_other] = 0

    # Data loader
    trainloader = torch.utils.data.DataLoader(dataset=mnist,
                                              batch_size=batch_size, 
                                              shuffle=True)

    import torch.nn.functional as F

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(nchannel, 3, 5)
            self.pool1 = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(3, 6, 5)
            self.pool2 = nn.MaxPool2d(2, 2)
            self.fc1 = nn.Linear(6 * 4 * 4, nclass)
            #self.fc2 = nn.Linear(44, 10)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.pool1(self.relu(self.conv1(x)))
            x = self.pool2(self.relu(self.conv2(x)))
            x = x.view(-1, 6 * 4 * 4)
            x = self.fc1(x)
            #x = self.relu(self.fc1(x))
            #x = self.fc2(x)
            return x

    net = Net()

    import torch.optim as optim

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    model_dir = './models'

    # Create a directory if not exists
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)


    for epoch in range(num_epochs):  # loop over the dataset multiple times

            running_loss = 0.0
            for i, data in enumerate(trainloader):
                #### get the inputs
                inputs, labels = data

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 2000 == 1999:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

    print('Finished Training')

    from torch.nn.utils import parameters_to_vector, vector_to_parameters

    one_parameter =  '/content/drive/My Drive/Keyword/8_real_'+str(seed)+'.pt'
    vec1 = parameters_to_vector(net.parameters())
    torch.save(vec1, one_parameter)
    print("Finished seed")
    print(jj)